In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import numpy as np 
import torch_geometric.nn as pyg_nn 
import torch_geometric.utils as pyg_utils
from torch_geometric.data import Dataset, Data, DataLoader
import torch.optim as optim
import os.path as osp
import scipy.io as sio
from datetime import datetime
from tensorboardX import SummaryWriter
from dataset import QUASARDataset
from model import ModelS

In [7]:
dir = '/home/hank/Datasets/QUASAR/small'
dataset = QUASARDataset(dir,num_graphs=100,remove_self_loops=True)
test_dir = '/home/hank/Datasets/QUASAR/small-test'
testset = QUASARDataset(test_dir,num_graphs=100,remove_self_loops=True)

Data graph type: 1.
Data graph type: 1.


In [8]:
GNN_TYPE = 'SAGE'
GNN_HIDDEN_DIM = 64
GNN_OUT_DIM = 64
GNN_LAYER = 7
NODE_MODE = 1
DATA_GRAPH_TYPE = 1
NUM_EPOCHES = 800
DROPOUT = 0.0
MLP_LAYER = 2
RESIDUAL = True
LOSSPOW = 1
model   = ModelS(node_feature_mode=1,
                    gnn_type=GNN_TYPE,
                    mp_hidden_dim=GNN_HIDDEN_DIM,mp_output_dim=GNN_OUT_DIM,mp_num_layers=GNN_LAYER, 
                    primal_node_mlp_hidden_dim=GNN_OUT_DIM,primal_node_mlp_output_dim=10,
                    dual_node_mlp_hidden_dim=GNN_OUT_DIM,dual_node_mlp_output_dim=10,
                    primal_edge_mlp_hidden_dim=GNN_OUT_DIM,primal_edge_mlp_output_dim=10, 
                    dual_edge_mlp_hidden_dim=GNN_OUT_DIM,dual_edge_mlp_output_dim=6,
                    primal_mlp_num_layers=MLP_LAYER, 
                    dual_mlp_num_layers=MLP_LAYER, 
                    dropout_rate=DROPOUT,
                    relu_slope=0.1,
                    residual=RESIDUAL,
                    losspow=LOSSPOW)
model.load_state_dict(torch.load('./models/shared_model_small_SAGE_7_True.pth'))
model.double()
model.eval()

Model: node_feature_mode = 1, mp_input_dim = 6, relu_slope = 0.1. GNN type: SAGE. Residual: True. Losspow: 1.


ModelS(
  (mp_convs): ModuleList(
    (0): SAGEConv(6, 64)
    (1): SAGEConv(64, 64)
    (2): SAGEConv(64, 64)
    (3): SAGEConv(64, 64)
    (4): SAGEConv(64, 64)
    (5): SAGEConv(64, 64)
    (6): SAGEConv(64, 64)
    (7): SAGEConv(64, 64)
    (8): SAGEConv(64, 64)
  )
  (primal_node_mlp): ModuleList(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=64, bias=True)
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): Linear(in_features=64, out_features=10, bias=True)
  )
  (dual_node_mlp): ModuleList(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=64, bias=True)
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): Linear(in_features=64, out_features=10, bias=True)
  )
  (primal_edge_mlp): ModuleList(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=64, bias=True)
    (2): Linear(in_featu

In [9]:
loader = DataLoader(dataset,batch_size=1,shuffle=True)
train_p_loss = []
train_d_loss = []
for batch in loader:
    x, X, S, Aty = model(batch)
    primal_loss, dual_loss = model.pd_loss(batch,X,S,Aty)
    train_p_loss.append(primal_loss.item())
    train_d_loss.append(dual_loss.item())
print('Train primal acc: {:.4f}'.format(torch.mean(torch.tensor(train_p_loss))))
print('Train dual acc: {:.4f}.'.format(torch.mean(torch.tensor(train_d_loss))))

loader = DataLoader(testset,batch_size=1,shuffle=True)
test_p_loss = []
test_d_loss = []
for batch in loader:
    x, X, S, Aty = model(batch)
    primal_loss, dual_loss = model.pd_loss(batch,X,S,Aty)
    test_p_loss.append(primal_loss.item())
    test_d_loss.append(dual_loss.item())
print('Test primal acc: {:.4f}'.format(torch.mean(torch.tensor(test_p_loss))))
print('Test dual acc: {:.4f}.'.format(torch.mean(torch.tensor(test_d_loss))))

/home/hank/miniconda3/envs/gnnsdp/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Train primal acc: 0.0399
Train dual acc: 0.3387.
Test primal acc: 1.0451
Test dual acc: 0.3527.


In [11]:
randmodel = ModelS(node_feature_mode=1,
                    gnn_type=GNN_TYPE,
                    mp_hidden_dim=GNN_HIDDEN_DIM,mp_output_dim=GNN_OUT_DIM,mp_num_layers=GNN_LAYER, 
                    primal_node_mlp_hidden_dim=GNN_OUT_DIM,primal_node_mlp_output_dim=10,
                    dual_node_mlp_hidden_dim=GNN_OUT_DIM,dual_node_mlp_output_dim=10,
                    primal_edge_mlp_hidden_dim=GNN_OUT_DIM,primal_edge_mlp_output_dim=10, 
                    dual_edge_mlp_hidden_dim=GNN_OUT_DIM,dual_edge_mlp_output_dim=6,
                    primal_mlp_num_layers=MLP_LAYER, 
                    dual_mlp_num_layers=MLP_LAYER, 
                    dropout_rate=DROPOUT,
                    relu_slope=0.1,
                    residual=RESIDUAL,
                    losspow=LOSSPOW)
randmodel.double()
randmodel.eval()

loader = DataLoader(dataset,batch_size=1,shuffle=True)
train_p_loss = []
train_d_loss = []
for batch in loader:
    x, X, S, Aty = randmodel(batch)
    primal_loss, dual_loss = randmodel.pd_loss(batch,X,S,Aty)
    train_p_loss.append(primal_loss.item())
    train_d_loss.append(dual_loss.item())
print('Rand train primal acc: {:.4f}'.format(torch.mean(torch.tensor(train_p_loss))))
print('Rand train dual acc: {:.4f}.'.format(torch.mean(torch.tensor(train_d_loss))))

loader = DataLoader(testset,batch_size=1,shuffle=True)
test_p_loss = []
test_d_loss = []
for batch in loader:
    x, X, S, Aty = randmodel(batch)
    primal_loss, dual_loss = randmodel.pd_loss(batch,X,S,Aty)
    test_p_loss.append(primal_loss.item())
    test_d_loss.append(dual_loss.item())
print('Rand test primal acc: {:.4f}'.format(torch.mean(torch.tensor(test_p_loss))))
print('Rand test dual acc: {:.4f}.'.format(torch.mean(torch.tensor(test_d_loss))))


Model: node_feature_mode = 1, mp_input_dim = 6, relu_slope = 0.1. GNN type: SAGE. Residual: True. Losspow: 1.


/home/hank/miniconda3/envs/gnnsdp/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Rand train primal acc: 1.2300
Rand train dual acc: 1.0000.
Rand test primal acc: 1.2333
Rand test dual acc: 1.0000.
